# Profiling

In this jupyter notebook, we will be profiling one of packages defined below

In [1]:
import importlib

# Imports
import time
import logging

import RabbitServerInfo as info
import DockerRemoteAPI as api
from ELSbeat import metricbeat
import LoadTester
import Manager

# for jupyter problems
importlib.reload(LoadTester)
importlib.reload(api)
importlib.reload(info)
importlib.reload(Manager)

from Manager import PackageInfo, ApiServer

# logging
logging.basicConfig(level=logging.FATAL)
LOGGER = logging.getLogger()

# Data Analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
profiler = ApiServer(name='profiler1', ip='10.2.6.177', api_port='7999', id='hkg-profiler-1')
workers = [
    ApiServer(name='profiler1', ip='10.2.6.177', api_port='7999', id='hkg-profiler-1'),
]

package_oltp = PackageInfo(image_name='YOUR_HUB_ID/oltp-automated',
                      test_url='http://10.2.6.171/oltp', mem_limit="512m", cpu_percent_limit=50, post_delay=300,
                      device_read_bps=[{"Path": "/dev/sda", "Rate": 7497318}],
                      device_write_bps=[{"Path": "/dev/sda", "Rate": 7497318}],)
package_fileio = PackageInfo(image_name='YOUR_HUB_ID/fileio-automated',
                      test_url='http://10.2.6.171/fileio', mem_limit="512m", cpu_percent_limit=50, post_delay=60,
                      device_read_bps=[{"Path": "/dev/sda", "Rate": 7497318}],
                      device_write_bps=[{"Path": "/dev/sda", "Rate": 7497318}],)
package_cpu = PackageInfo(image_name='YOUR_HUB_ID/cpu-automated',
                      test_url='http://10.2.6.171/linpack', mem_limit="512m", cpu_percent_limit=50, post_delay=30,
                      device_read_bps=[{"Path": "/dev/sda", "Rate": 7497318}],
                      device_write_bps=[{"Path": "/dev/sda", "Rate": 7497318}],)

manager = Manager.Manager(profiler=profiler, workers=workers, elastic_server_ip='10.2.6.181', elastic_server_port='9200')

In [3]:
warmup_time = '10s'
test_time = '1m'
accepted_stats=['avg']

test_mode = "Interactive"

if test_mode == "RPS":
    profile_targets = [
        {
            'package': package_cpu,
            'target_rps_list': [1,2,3],
        },
        {
            'package': package_fileio,
            'target_rps_list': [1,2,3],
        },
        {
            'package': package_oltp,
            'target_rps_list': [2,4,6],
        }
    ]
elif test_mode == "Interactive":
    profile_targets = [
        {
            'package': package_cpu,
            'num_of_clients': [1,2,3],
        },
        {
            'package': package_fileio,
            'num_of_clients': [1,3,5],
        },
        {
            'package': package_oltp,
            'num_of_clients': [1,2,4,6],
        }
    ]

In [4]:
# manager.prepare_profile(package=package_fileio)

In [ ]:
if test_mode == "RPS":
    for profiling in profile_targets:
        # Prepare the profiler server
        manager.prepare_profile(package=profiling['package'])
        profiling['all_results'] = []

        for target_rps in profiling['target_rps_list']:
            print("Starting profiling for ", profiling['package'], " with RPS of ", target_rps)
            profile_results = manager.profile(package=profiling['package'], warmup_time=warmup_time, test_time=test_time,
                        accepted_stats=accepted_stats, target_rps=target_rps, count=3)
            profile_results['target_rps'] = target_rps
            profiling['all_results'].append(profile_results)
elif test_mode == "Interactive":
    for profiling in profile_targets:
        # Prepare the profiler server
        manager.prepare_profile(package=profiling['package'])
        profiling['all_results'] = []

        for num_of_clients in profiling['num_of_clients']:
            print("Starting profiling for ", profiling['package'], " with ", num_of_clients, " clients")
            profile_results = manager.profile(package=profiling['package'], warmup_time=warmup_time, test_time=test_time,
                        accepted_stats=accepted_stats, target_rps=num_of_clients, count=10, test_mode=test_mode)
            profile_results['num_of_clients'] = num_of_clients
            profiling['all_results'].append(profile_results)

In [6]:
cpu_percent = lambda x: (x['cpu_time-avg'] - x['cpu_idle-avg'] - x['cpu_io_wait-avg'])/x['cpu_time-avg']*100
success_ratio = lambda x: x['completed'] / (x['completed'] + x['failed'])
if test_mode == "RPS":
    inv_priority = lambda x: (x['avg_ms']+1)/(x['success_ratio'] + .001) # add with 0.001 to fix success ratio of zero
elif test_mode == "Interactive":
    inv_priority = lambda x: (x['avg_ms']+1)/(x['success_ratio'] + .001)/(x['completed'] + .001) # add with 0.001 to fix success ratio of zero

if test_mode == "RPS":
    final_table = []
    for profiling in profile_targets:
        for idx, results in enumerate(profiling['all_results']):
            res = pd.DataFrame(data=results)
            res['cpu_percent'] = res.apply(cpu_percent, axis=1)
            res['success_ratio'] = res.apply(success_ratio, axis=1)
            res['inv_priority'] = res.apply(inv_priority, axis=1)

            best_res_idx = res['inv_priority'].idxmin()

            selected_row = res.iloc[best_res_idx,:].copy()
            selected_row['target_rps'] = profiling['target_rps_list'][idx]
            selected_row['package_name'] = profiling['package'].image_name
            selected_row['package_mem_limit'] = profiling['package'].mem_limit
            selected_row['package_device_read_bps'] = profiling['package'].device_read_bps
            selected_row['package_device_read_iops'] = profiling['package'].device_read_iops
            selected_row['package_device_write_bps'] = profiling['package'].device_write_bps
            selected_row['package_device_write_iops'] = profiling['package'].device_write_iops
            selected_row['package_cpu_percent_limit'] = profiling['package'].cpu_percent_limit
            selected_row['package_test_url'] = profiling['package'].test_url
            selected_row['package_post_delay'] = profiling['package'].post_delay

            final_table.append(selected_row.to_dict())
elif test_mode == "Interactive":
    final_table = []
    for profiling in profile_targets:
        for idx, results in enumerate(profiling['all_results']):
            res = pd.DataFrame(data=results)
            res['cpu_percent'] = res.apply(cpu_percent, axis=1)
            res['success_ratio'] = res.apply(success_ratio, axis=1)
            res['inv_priority'] = res.apply(inv_priority, axis=1)

            best_res_idx = res['inv_priority'].idxmin()

            selected_row = res.iloc[best_res_idx,:].copy()
            selected_row['num_of_clients'] = profiling['num_of_clients'][idx]
            selected_row['package_name'] = profiling['package'].image_name
            selected_row['package_mem_limit'] = profiling['package'].mem_limit
            selected_row['package_device_read_bps'] = profiling['package'].device_read_bps
            selected_row['package_device_read_iops'] = profiling['package'].device_read_iops
            selected_row['package_device_write_bps'] = profiling['package'].device_write_bps
            selected_row['package_device_write_iops'] = profiling['package'].device_write_iops
            selected_row['package_cpu_percent_limit'] = profiling['package'].cpu_percent_limit
            selected_row['package_test_url'] = profiling['package'].test_url
            selected_row['package_post_delay'] = profiling['package'].post_delay

            final_table.append(selected_row.to_dict())


            
final_frame = pd.DataFrame(data=final_table)        
final_frame.to_csv('ProfileTable_' + test_mode + '.csv')
        
        

In [ ]:
# Look at final frame
final_frame